### 7장 텍스트 데이터 다루기

In [15]:
sentences = ['가장 자연스럽고 좋은 좋은 ~ 영화 였다',
            '최고의 영화 4편이 기대됨']

In [16]:
type(sentences), len(sentences)

(list, 2)

##### BOW(Bag of Words)  단어 가방으로 전처리 

In [17]:
#420
from sklearn.feature_extraction.text import CountVectorizer #갯수를 세어서 배열로 만든다

In [18]:
vect = CountVectorizer()

In [19]:
vect.fit(sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [20]:
len(vect.vocabulary_), vect.vocabulary_ #등장하는 어휘들 

(8,
 {'가장': 1,
  '자연스럽고': 5,
  '좋은': 6,
  '영화': 4,
  '였다': 3,
  '최고의': 7,
  '4편이': 0,
  '기대됨': 2})

In [21]:
#dense_matrix <-> sparse_matrix(자연어): 메모리 절약하기 위해서 사용
sparse_matrix = vect.transform(sentences) #자연어를 BOW 로 변환
sparse_matrix

<2x8 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [22]:
sparse_matrix.toarray() # sparse 를 dense 로 변환

array([[0, 1, 0, 1, 1, 1, 2, 0],
       [1, 0, 1, 0, 1, 0, 0, 1]], dtype=int64)

In [23]:
sentences

['가장 자연스럽고 좋은 좋은 ~ 영화 였다', '최고의 영화 4편이 기대됨']

In [24]:
from sklearn.datasets import fetch_20newsgroups

In [26]:
news = fetch_20newsgroups() # Download 진행됩니다

In [33]:
len(news.data), len(news.target_names)

(11314, 20)

In [44]:
news.target[2], news.target_names[2]

(4, 'comp.os.ms-windows.misc')

자연어 뉴스를 배열(vector)로 변환 / BOW

In [46]:
vect = CountVectorizer().fit(news.data) #

In [49]:
len(vect.vocabulary_)

130107

In [50]:
sm = vect.transform(news.data)

In [51]:
sm

<11314x130107 sparse matrix of type '<class 'numpy.int64'>'
	with 1787565 stored elements in Compressed Sparse Row format>

In [52]:
from sklearn.linear_model import LogisticRegression

In [54]:
model = LogisticRegression().fit(sm, news.target)

C:\Users\wtime\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\wtime\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [66]:
model.score(sm, news.target)

0.999734841788934

In [70]:
test_set = ['Hi, Major League, who is winner? ', 'I want to buy a notebook. where shoud I go? ']

In [71]:
model.predict(vect.transform(test_set))

array([10,  4])

In [73]:
news.target_names[9], news.target_names[4]

('rec.sport.baseball', 'comp.sys.mac.hardware')

In [75]:
sm_test = vect.transform(test_set)
dm = sm_test[0].toarray()
dm.shape

(1, 130107)

In [80]:
sm_test[0].indices #indexes 배열의 크기가 너무 크기 때문에 0이 아닌 값이 있는 위치를 볼 수 밖에 없습니다

array([ 63092,  68532,  75022,  79043, 124147, 124501])

In [78]:
np.sum(dm), dm

(6, array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64))

In [81]:
import konlpy

In [82]:
from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()
pprint(kkma.sentences(u'네, 안녕하세요. 반갑습니다.'))

C:\Users\wtime\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


['네, 안녕하세요.', '반갑습니다.']


In [91]:
#p445
from konlpy.tag import Okt
okt_tag = Okt()
okt_tag.morphs('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.')

['질문', '이나', '건의', '사항', '은', '깃', '헙', '이슈', '트래커', '에', '남겨주세요', '.']

In [168]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = TfidfVectorizer(tokenizer=okt_tag.morphs) # 문장 > 벡터 로 변환할때 단어 자르는 기능을 konlpy를 사용

In [116]:
sentences = ['안녕, 나는 남자야 남자!', '안녕, 나는 여자야 여자!!'] 

In [117]:
cv = cv.fit(sentences) # 토큰(단어)을 다 잘라서 어휘 사전(vocaburary) 를 준비합니다

In [118]:
cv.vocabulary_

{'안녕': 6, ',': 2, '나': 3, '는': 5, '남자': 4, '야': 7, '!': 0, '여자': 8, '!!': 1}

In [119]:
sparse_matrix = cv.transform(sentences) # 문장을 희소 행렬로 변환

In [120]:
sparse_matrix.toarray()

array([[1, 0, 1, 1, 2, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 1, 1, 1, 2]], dtype=int64)

In [83]:
pprint(kkma.nouns(u'질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [85]:
kkma.morphs(u'오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^')

['오류',
 '보고',
 '는',
 '실행',
 '환경',
 ',',
 '에러',
 '메세지',
 '와',
 '함께',
 '설명',
 '을',
 '최대한',
 '상세히',
 '!',
 '^^']

In [84]:
pprint(kkma.pos(u'오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))

[('오류', 'NNG'),
 ('보고', 'NNG'),
 ('는', 'JX'),
 ('실행', 'NNG'),
 ('환경', 'NNG'),
 (',', 'SP'),
 ('에러', 'NNG'),
 ('메세지', 'NNG'),
 ('와', 'JKM'),
 ('함께', 'MAG'),
 ('설명', 'NNG'),
 ('을', 'JKO'),
 ('최대한', 'NNG'),
 ('상세히', 'MAG'),
 ('!', 'SF'),
 ('^^', 'EMO')]


네이버 영화 리뷰 추천/비추천 분류 

In [122]:
import pandas as pd

C:\Users\wtime\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\wtime\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [125]:
movie = pd.read_csv('ratings_train.txt', delimiter='\t')
movie.shape

(150000, 3)

In [128]:
movie = movie.drop(columns='id')
movie = movie.dropna()

In [131]:
m = 1000
movie = movie.iloc[:m]

In [133]:
movie.head(2)

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1


In [137]:
movie.isna().sum()

document    0
label       0
dtype: int64

In [139]:
X = movie['document']
y = movie.label

In [140]:
from konlpy.tag import Okt
cv = CountVectorizer(tokenizer= Okt().morphs) # 전처리기

In [170]:
cv = cv.fit(X)

In [171]:
len(cv.vocabulary_)

4767

In [172]:
X_sm = cv.transform(X)

In [173]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier # classification

In [174]:
scores = cross_val_score(RidgeClassifier(), X_sm, y, cv=3)
scores

array([0.74550898, 0.73573574, 0.68768769])

In [175]:
model = RidgeClassifier().fit(X_sm, y)

In [176]:
sentences = ['두시간동안 잘 자고 나왔습니다', '강력 수면제!! ㅋㅋ ', 
             '돈이 너무 아까워요 ㅜㅜ', '너무 재밌어요 ㅋ ', '두번 봐라 ']
model.predict(cv.transform(sentences))

array([0, 1, 0, 1, 0], dtype=int64)